In [ ]:
import numpy as np
import pandas as pd
import scipy as sc
import os
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import statsmodels.formula.api as sm
import seaborn as sns

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 10)
plt.style.use('ggplot')

In [ ]:
import json, pickle
from ipyleaflet import Map, basemaps, GeoJSON, Popup, FullScreenControl, CircleMarker, LayerGroup
# if above module isn't installed, do BOTH of the following:
# pip install ipyleaflet
# jupyter nbextension enable --py --sys-prefix ipyleaflet

from ipywidgets import HTML

In [ ]:
origdir = os.getcwd()
os.chdir('..')

In [ ]:
# Read in scraped car data
data_file = os.path.join(os.getcwd(),"data","all_cars.csv")  
cars = pd.read_csv(data_file)
# cars = pd.read_pickle('cars.pkl')
pd.set_option("display.max_rows",None,"display.max_columns",None) 

# Recast data
cars['mileage'] = cars['mileage'].astype('float')
cars['year'] = cars['year'].astype('float')

# Clean price data and recast
ugly_cars = cars[cars['price'].str.contains('MSRP')]
ugly_cars.index
for index, car in ugly_cars.iterrows():
    if '|' not in car['price']:
        cars.at[index,'price'] = None
    else:
        cars.at[index,'price'] = car['price'].split('|')[0].strip()
cars['price'] = cars['price'].astype('float')

In [ ]:
os.chdir(origdir)

In [ ]:
# Data clean up
# Check year
# any cars with a year less than 1920 changed to None
cars.loc[(cars.year < 1920),'year']=None 

# Check price
# any cars with a price less than $100 or greater than $500,000 changed to None
cars.loc[(cars.price < 100),'price']=None 
cars.loc[(cars.price > 500000),'price']=None 

# Check Mileage
# any cars with a mileage greater than 500000 changed to None
cars.loc[(cars.mileage > 500000),'mileage']=None 

# Check Liters
# any cars with a mileage greater than 500000 changed to None
cars.loc[(cars.liters > 100),'liters']=None

# Check fav_per_view
# any cars with inf or Nan changed to None
cars.loc[(cars.fav_per_view > 2),'fav_per_view']=None 

In [ ]:
cars.head(15)

In [ ]:
# Add in a city, state column
cars['citystate_abb'] = cars[['city', 'state']].apply(lambda x: ', '.join(x.astype(str)), axis=1)

# only get non-null expected price rows
good_cars = cars[cars.expected_price.notnull()]

In [ ]:
# load zip code coordinates
with open('zip_coord_api.pkl','rb') as handle:
    zip_coord = pickle.load(handle)

In [ ]:
# reverse the ZIP:lat/long dictionary
coor_zip = {str(round(v[0], 3))+str(round(v[1], 3)): k for k, v in zip_coor.items()}

if len(coor_zip) != len(zip_coor):
    raise ValueError('the reversed ZIP code dictionary had a different number of keys than the original')

In [ ]:
import random
random.randint(0,4)
colors = ['red', 'orange', 'green']

In [ ]:
utah_center = [39.3210, -111.0937]
zoom = 6
m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=utah_center, zoom=zoom)
m.add_control(FullScreenControl())

# create a layer group 
layer_group = LayerGroup()
for zipp, coord in zip_coord.items():
    circle = CircleMarker()
    circle.location = coord
    circle.radius = random.randint(1,10)
    circle.weight = 2
    circle.opacity = 0.8
    color = random.randint(0,2)
    circle.color = colors[color]
    circle.fill_color = colors[color]
    circle.fill_opacity = 0.3
    layer_group.add_layer(circle)
    

m.add_layer(layer_group)
m

In [ ]:
# use mouseover and mouseout events in CircleMarker class

# grab a few locations and associated data
loc0 = good_cars.loc[cars['city'] == 'Salt Lake City'].zip_code.mode()[0]
loc1 = good_cars.loc[cars['city'] == 'Sandy'].zip_code.mode()[0]
loc2 = good_cars.loc[cars['city'] == 'West Jordan'].zip_code.mode()[0]
print(f'coordinates for {loc0} are {zip_coor[loc0]}')
print(f'coordinates for {loc1} are {zip_coor[loc1]}')
print(f'coordinates for {loc2} are {zip_coor[loc2]}')
subdf0 = cars.loc[cars['zip_code'] == loc0]
subdf1 = cars.loc[cars['zip_code'] == loc1]
subdf2 = cars.loc[cars['zip_code'] == loc2]

# from ipyleaflet import Map, basemaps, GeoJSON, Popup, FullScreenControl, CircleMarker, MarkerCluster
# from traitlets import Instance, Dict, List, Int, Unicode
# from ipywidgets import CallbackDispatcher, HTML

center = [40.7608, -111.8910] # SLC, UT
zoom = 8
m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=zoom)
m.add_control(FullScreenControl())

def hover_on(event, type, coordinates):
    # use coordinates to reverse look up data associated with those coordinates
    currzip = coor_zip[str(round(coordinates[0], 3))+str(round(coordinates[1], 3))]
    currdata = cars.loc[cars['zip_code'] == currzip]
    currlabel = currdata.citystate_abb.mode()[0]
    currtotal = "{:,}".format(currdata.shape[0])
    currmedprice = "{:,}".format(int(currdata.price.median()))
    currmedmile = "{:,}".format(int(currdata.mileage.median()))
    currmedyear = str(int(currdata.year.median()))
    currcommcar = str(currdata.make.mode()[0]) + ' ' + str(currdata.model.mode()[0])
    
    # remove old popup layer
    if isinstance(m.layers[-1], Popup):
        m.remove_layer(m.layers[-1])

    # add a popup layer on hover over a city
    message = HTML()

    message.value = ('<h4><strong>' + currlabel + '</strong></h4>' +
                     '<table>' +
                         '<tr><td>Total Cars:&emsp;</td><td>' + currtotal + '</td></tr>' +
                         '<tr><td>Median Price:&emsp;</td><td>$' + currmedprice + '</td></tr>' +
                         '<tr><td>Median Mileage:&emsp;</td><td>' + currmedmile + '</td></tr>' +
                         '<tr><td>Median Year:&emsp;</td><td>' + currmedyear + '</td></tr>' +
                         '<tr><td>Most Common Car:&emsp;</td><td>' + currcommcar + '</td></tr>' +
                     '</table>')
    
    popup = Popup(location=coordinates, child=message, close_button=False, auto_close=True, close_on_escape_key=False)

    m.add_layer(popup) # add the new layer

circobj0 = CircleMarker(location = zip_coor[loc0])
circobj0.on_mouseover(hover_on)

circobj1 = CircleMarker(location = zip_coor[loc1])
circobj1.on_mouseover(hover_on)

circobj2 = CircleMarker(location = zip_coor[loc2])
circobj2.on_mouseover(hover_on)

m.add_layer(circobj0)
m.add_layer(circobj1)
m.add_layer(circobj2)
m